In [ ]:
!nvidia-smi

# Import libraries 

In [1]:
import os

In [4]:
os.chdir('/home/ali/RELAXIN/alphafold')

In [5]:
ls

rec_relax.pdb


# Prepare the system

In [ ]:
!martinize2 -f model_start_aligned.pdb -ff martini3001 -x start_cg.pdb -o topoltemp.top -dssp /usr/bin/dssp -elastic -scfix -cys 0.40 -p backbone -pf 1000 -nt

In [ ]:
!python2.7 insane_altered.py -salt 0.15 -charge -0 -sol W -f start_cg.pdb -o system.gro -pbc rectangular -d 30 -x 9 -y 9 -z 12 -l POPC:100 -p topol.top

In [ ]:
!sed -i -e "s/Protein        1/molecule_0 1\nmolecule_1 1/g" ./topol.top

In [ ]:
!cat topol.top

# Energy Minimization

In [ ]:
!gmx_mpi grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.0_minimization -ntomp 5 -v 

In [ ]:
!gmx_mpi grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.1_minimization -ntomp 5 -v

# Equilibration Steps - NVT

In [ ]:
!gmx_mpi grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.2_equilibration -ntomp 5 -gpu_id 0 -v

# Equilibration Steps - NPT

In [ ]:
!gmx_mpi grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.3_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
!gmx_mpi grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.4_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
!gmx_mpi grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.5_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
!gmx_mpi grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.6_equilibration -ntomp 5 -gpu_id 0 -v

# MD run 

In [ ]:
!gmx_mpi grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step7_production -ntomp 5 -gpu_id 0 -v

In [ ]:
!gmx_mpi check -f step7_production.xtc

In [ ]:
ls

# make an Index

In [ ]:
#get the number of the first and last atoms of the receptor and peptide
!cat system.gro
#!cat system.gro

In [ ]:
#cg
!echo -e "a 1-725  \nname 22 rec \na 726-750 \nname 23 pep \nq" |gmx make_ndx -f system.gro -o index.ndx

# Center the the box
    - We remove PBC
    - output = step7_production.pbc.xtc

In [ ]:
ls *trr

In [ ]:
!echo 17 0|gmx_mpi trjconv -f step7_production.xtc -o step7_production.pbc.xtc -s step7_production.tpr -pbc mol -center -n index.ndx 

# Trajectory Alignment
    - We align the traj to the CG system to remove translation and rotation
    - output = traj_aligned_to_CG.trr

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import contacts, align

finp_ref  = 'system.gro'                     # reference
finp_trj  = 'step7_production.pbc.xtc'       # traj


receptor = 'name BB and resid 40-337'


U   = mda.Universe(finp_ref, finp_trj)       #Open Traj
ref = mda.Universe(finp_ref)                 #Open ref file 

# Align traj to reference using atmgroup receptor
align.AlignTraj( U, ref, select=receptor, in_memory=True).run()
'Done Boss'

# save aligned traj
system_aligned = U.select_atoms("all")
with mda.Writer("traj_aligned_to_CG.trr", system_aligned.n_atoms) as W:
    for ts in U.trajectory:
        W.write(system_aligned)

# NOW READY TO BE ANALYZED!!